In [ ]:
from AMC_python import AMC
from TimeTagger import *
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime
import pandas as pd


# Initialize TCSPC
tt = createTimeTagger()


def histo(t,rep_rate):
"""
Function for measuring lifetime (histogram from the SPAD channel such that start clicks are taken from 
sync channel and stop clicks from the SPAD channel of the TCSPC)
Arg: t - acqusition time in picoseconds; rep_rate - repetition rate of the pulsed laser
Return: numpy.array of the histogram with length = no. of bins
Displays real-time output of the measurement 
Based on the Histogram measurement class of Swabian API
"""
    SYNC_CH = 3 # Electrical trigger from pulsed laser
    SPD_CH = 1  # SPAD channel 1
    delay = 18e3 # Controllable delay from SPAD channel
    tt.setInputDelay(SPD_CH, delay)
    n_values = 2e4/rep_rate # total no. of bins of the histogram
    binwidth = 50 # binwidth of histogram bins in ps
    chist = Histogram(tt, SPD_CH, SYNC_CH, binwidth,n_values) # Histogram measurement class of Swabian API
    chist.startFor(t) # start acquisition
    while chist.isRunning():
        arr = chist.getData() # Acquire data
        td = [i*binwidth/1e3 for i in range(len(arr))] # define time array (histogram bins)
        # Real-time display
        clear_output(wait=True)
        plt.plot(td,arr[range(len(arr))])
        plt.show()
    chist.stop()
    return arr # Returns the histogram

        
def cntrate(t,x):
"""
Function for measuring countrate (counts/s or cps) for 'x' channel
Args: t - acqusition time in picoseconds; x - channel no.
Return: numpy.array of len 1 containing countrate from channel x
Displays real-time output of the measurement 
Based on the Countrate measurement class of Swabian API
"""
    cr = Countrate(tt, [x])  # Measurement class of Swabian API
    cr.startFor(t) # Set acquisition time
    
    while cr.isRunning():
        countrates = cr.getData() # Acquire countrate data
        print(countrates, end='\r')
    return countrates # Returns countrate from x

# IP Address of the AMC driver
IP = "192.168.1.1"

# Setup connection to AMC
amc = AMC.Device(IP)
amc.connect()

# Activate axes
axis0 = 0 # Axis 1
amc.control.setControlOutput(axis0, True)
axis1 = 1 # Axis 2
amc.control.setControlOutput(axis1, True)



def raster_scan(x_move,y_move,x_step,y_step,thresh,modehist,t,rep_rate):
"""
Function for performing scanning lifetime measurements
Args: x_move, y_move = total no. of steps to move in x, y directions, respectively
x_step, y_step = no. of steps to move in x or y direction with one iteration (x,y resolution)
t = acquisition time of lifetime function
modehist = Boolean; whether to conduct histogram measurement with scan or not
thresh = intensity threshold in cps above which lifetime will be measured
rep_rate = Repetition rate of the laser 

Return: list of tuples: (x, y, countrate, histogram numpy.array);
at pixels where lifetime is not measured, zeroes are returned
Displays real-time output of the measurement (plots intensity matrix)
"""
    
    pos_xy = [] # Initialize list of tuples
    cnts = np.zeros((y_move,x_move+1))  # Initialize counts map plot to zeroes
    amc.control.setControlAmplitude(axis1,25000) # y axis voltage is applied

    plt.subplots(1,2,figsize=(15,15))
    for i in range(y_move): # For loop for y motion
        x = i
        amc.move.setNSteps(axis1, False, y_step) ## Moves Up == False
                                                 ## Moves Down == True

        histy = np.zeros(int(1e4/rep_rate)*100) # Initialize lifetime np.array to zeroes of len == no. of bins
        
        cnty = cntrate(0.1e12,1) # Acquire counts from y position for 0.1e12 seconds
        if modehist == True and cnty > thresh:
            histy = histo(t,rep_rate) # Measure histogram
            time.sleep(2)
            # Clear the overflows to get correct measurement values for subsequent calls of the function
            for _ in range(100):
                tt.getOverflowsAndClear()
            
        # Reshape matrix to plot in real time and mimic raster scan motion
        if x%2 == 1:
            cnts[i,x_move] = cnty
        else:
            cnts[i,0] = cnty
            
        # Real-time display
        clear_output(wait=True)
        img = plt.imshow(cnts)
        plt.colorbar(img)
        plt.show()
        # Store x, y position, countrate, lifetime np.array whwerever measured (if not measured histogram array is made of zeroes)        
        pos_xy.append((amc.move.getPosition(axis0),amc.move.getPosition(axis1),histy,cnty))
        

        # For loop for x motion
        for j in range(x_move):
            # IF x motion in fwd and back direction is not the same for a particular amplitude of actuator signal
            if x%2 == 1:
                ### BACKWARD
                amc.control.setControlAmplitude(axis0,22600) ## Changes amplitude in back (x avis) 
            else:
                ### FORWARD
                amc.control.setControlAmplitude(axis0,27301)  ## Changes amplitude in fwd (x avis) 
                
            amc.move.setNSteps(axis0, x%2, x_step)  ## Moves in x
            
            histx = np.zeros(int(1e4/rep_rate)*100) # Initialize lifetime np.array to zeroes of len == no. of bins
            
            cntx = cntrate(0.1e12,1) # Acquire counts from x position for 0.1e12 seconds
            if mode == True and cntx > thresh:
                histx = histo(t,rep_rate)
                time.sleep(2)
                # Clear the overflows to get correct measurement values for subsequent calls of the function
                for _ in range(100):
                    tt.getOverflowsAndClear()
                    
            # Store x, y position, countrate, lifetime np.array whwerever measured (if not measured histogram array is made of zeroes)
            pos_xy.append((amc.move.getPosition(axis0),amc.move.getPosition(axis1),histx,cntx))

            
            # Reshape matrix to plot in real time and mimic raster scan motion
            if x%2 == 0:
                cntr[i,j+1] = cntx
            else:
                cntr[i,x_move-(j+1)] = cntx              
                
            # Real-time display
            clear_output(wait=True)
            img = plt.imshow(cnts)
            plt.colorbar(img)
            plt.show()

    
    return pos_xy # Returns final list of tuples



x_step = 1 # x step
y_step = 1 # y step
thresh = 120000 # threshold intensity
modehist = False # mode of measurement incl. lifetime or not
rep_rate = 20 # in MHz 
t_hist = 30e12 # lifetime acquisition time in ps
y_move = int(input("No. of steps in y:")) # user input for total x steps
x_move = int(input("No. of steps in x:")) # User input for total y steps
save = input("Do you want to save the xy data for this scan (y or n)?") 


p = raster_scan(x_move,y_move,x_step,y_step,thresh,modehist,t_hist,rep_rate) # measurement data stored in p


# Saving protocol
date = datetime.today().strftime('%Y%m%d') 
folder_name = datetime.today().strftime('%B-%Y') 
if save == 'y':
    scan_num = int(input("Scan num:"))
    sample = input("Sample name (without space):")    
    df = pd.DataFrame(p, columns = ['X_data','Y_data','Lifetime','Counts'])
    path = 'D:\\Data\\'+folder_name+'\\'+date
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    df.to_csv(path+'\\'+sample+'_scan'+str(scan_num)+'_FLIM.csv', index=False)
    xy = [(i[0],i[1]) for i in p]
    print('Your file is saved as:',path+'\\'+sample+'_scan'+str(scan_num)+'_FLIM.csv')
    
    

